In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

items = pd.read_csv(r"..\Data\Raw\items.csv", delimiter=",")
print(items.head())
print(items.shape)
items = items.drop_duplicates(subset=['item_name'], keep=False)
print(items.shape)

                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
(22170, 3)
(22170, 3)


In [3]:
test = pd.read_csv(r"..\Data\Raw\test.csv", delimiter=",")
print(test.head())
print(test['shop_id'].nunique())
print(test['ID'].nunique())

   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
42
214200


In [4]:
sales = pd.read_csv(r"..\Data\Raw\sales_train.csv", delimiter=",")
print(sales.head())
print(sales.shape)
sales = sales.drop_duplicates(subset=['date', 'shop_id', 'item_id'], keep=False)
print(sales.shape)
sales = sales[sales.item_cnt_day<1000]
sales = sales[sales.item_price<60000]
sales = sales[sales.item_price>0]
print(sales.shape)
sales['revenue'] = sales['item_cnt_day'] * sales['item_price']
sales_monthly = sales.groupby(["date_block_num", "shop_id", "item_id"]).agg({'item_cnt_day': ['sum'], 'item_price':['median'], 'revenue':['sum']})
print(sales_monthly.head(20))
sales_monthly.sample(10)

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
(2935849, 6)
(2935793, 6)
(2935789, 6)
                               item_cnt_day item_price revenue
                                        sum     median     sum
date_block_num shop_id item_id                                
0              0       32               6.0      221.0  1326.0
                       33               3.0      347.0  1041.0
                       35               1.0      247.0   247.0
                       43               1.0      221.0   221.0
                       51               2.0      128.5   257.0
             

,,,item_cnt_day,item_price,revenue
,,,sum,median,sum
date_block_num,shop_id,item_id,,,
29,19,16138,1.0,679.0,679.0
11,7,6501,9.0,349.5,3145.5
4,52,177,1.0,250.0,250.0
9,35,19461,1.0,249.0,249.0
3,51,11711,5.0,207.0,1035.0
1,0,13517,7.0,2300.0,17573.0
11,25,9595,1.0,199.0,199.0
2,57,3743,2.0,559.0,1118.0


In [5]:
#join to test set to get only necessary IDs
month_range = range(1, 34)
month_df = pd.DataFrame(month_range)

month_df.rename(columns={0 :'date_block_num'}, inplace=True )
print(month_df.head())
month_df['temp'] = 1
test['temp'] = 1
test_monthly = test.merge(month_df, on=['temp'])
test_monthly.drop(['temp'], axis=1, inplace=True)
whole_df = test_monthly.merge(sales_monthly, how='left', on=['date_block_num', 'item_id', 'shop_id'])
whole_df.sample(10)
print(whole_df.shape)

   date_block_num
0               1
1               2
2               3
3               4
4               5
(7068600, 7)


e:\latest pycharm\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [6]:
categories = pd.read_csv(r"..\Data\Raw\item_categories.csv", delimiter=",")
print(categories.head())


        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4


In [7]:
print(items.shape)
print(categories.shape)
item_cats = items.merge(categories, how='left', on=['item_category_id'])
print(item_cats.shape)
print(item_cats.head())
item_cats.drop(['item_category_name', 'item_name'], axis=1, inplace=True)
print(item_cats.head())
print(item_cats['item_id'].nunique())
print(item_cats.shape)
whole_df = whole_df.merge(item_cats, how='left', on=['item_id'])
whole_df.sample(10)

(22170, 3)
(84, 2)
(22170, 4)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id                    item_category_name  
0                40                            Кино - DVD  
1                76  Программы - Для дома и офиса (Цифра)  
2                40                            Кино - DVD  
3                40                            Кино - DVD  
4                40                            Кино - DVD  
   item_id  item_category_id
0        0                40
1        1                76
2        2                40
3        3                40
4        4                40
22170
(22170, 2)


,ID,shop_id,item_id,date_block_num,"(item_cnt_day, sum)","(item_price, median)","(revenue, sum)",item_category_id
2895675,87747,18,13236,25,NaN,NaN,NaN,47
4180000,126666,52,1809,23,1.0,349.0,349.0,30
4195543,127137,52,1866,23,NaN,NaN,NaN,24
31663,959,5,16988,17,NaN,NaN,NaN,40
4302373,130374,47,7327,32,NaN,NaN,NaN,55
1204528,36500,12,13226,29,NaN,NaN,NaN,47
2156249,65340,22,22013,30,NaN,NaN,NaN,38
1095009,33182,10,892,4,NaN,NaN,NaN,77
510386,15466,3,10515,9,NaN,NaN,NaN,40
4153187,125854,52,14969,6,NaN,NaN,NaN,47


In [8]:
whole_df.fillna(0, inplace=True)
list(whole_df.columns)
whole_df.rename(columns={('item_cnt_day', 'sum'):'total_count', ('item_price', 'median'): 'median_price', ('revenue', 'sum'):'total_revenue'}, inplace=True )
whole_df['total_count'].clip(0,20)
print(whole_df.sample(10))
whole_df.to_csv('../Data/Processed/pre_normalised_whole_df.csv', index=False)

             ID  shop_id  item_id  date_block_num  total_count  median_price  \
5644900  171057       37    20736              20          0.0           0.0   
454855    13783        6    10776              17          0.0           0.0   
2320868   70329       24    15298              12          0.0           0.0   
2966289   89887       18    15503              19          0.0           0.0   
182116     5518        4     2326              23          0.0           0.0   
6061810  183691       34     4367               8          0.0           0.0   
4359889  132117       47     3418              29          0.0           0.0   
2634159   79823       15    11486               1          0.0           0.0   
6503934  197088       41    13853              31          0.0           0.0   
4406028  133516       48    21670               1          0.0           0.0   

         total_revenue  item_category_id  
5644900            0.0                72  
454855             0.0           

In [9]:
whole_df.sort_values(by=['ID', 'date_block_num'])
whole_df = whole_df[whole_df['item_id'] <= 1000]
whole_df.drop(['ID', 'total_revenue'], axis=1, inplace=True)
whole_df = whole_df[['total_count', 'date_block_num', 'shop_id', 'item_id', 'item_category_id', 'median_price']]
print(whole_df.head(34))
whole_df_arr = whole_df.values
print(whole_df_arr.shape)
whole_df_arr = whole_df_arr.reshape(7350, 33, 6)
print(whole_df_arr)#change back to 214200


       total_count  date_block_num  shop_id  item_id  item_category_id  \
15774          0.0               1        5      560                77   
15775          0.0               2        5      560                77   
15776          0.0               3        5      560                77   
15777          0.0               4        5      560                77   
15778          0.0               5        5      560                77   
15779          0.0               6        5      560                77   
15780          0.0               7        5      560                77   
15781          0.0               8        5      560                77   
15782          1.0               9        5      560                77   
15783          0.0              10        5      560                77   
15784          0.0              11        5      560                77   
15785          0.0              12        5      560                77   
15786          0.0              13    

In [11]:
#prepare training and validation sets
train_arr = whole_df_arr[:, :26]
val_arr = whole_df_arr[:, 26:]
print(train_arr.shape)
print(val_arr.shape)
train_arr_first_test = train_arr[:, 19:25]
print(train_arr_first_test.shape)
print(train_arr)

(7350, 26, 6)
(7350, 7, 6)
(7350, 6, 6)
[[[0.00e+00 1.00e+00 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.00e+00 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  [0.00e+00 3.00e+00 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  ...
  [0.00e+00 2.40e+01 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.50e+01 5.00e+00 5.60e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.60e+01 5.00e+00 5.60e+02 7.70e+01 0.00e+00]]

 [[0.00e+00 1.00e+00 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.00e+00 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  [0.00e+00 3.00e+00 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  ...
  [0.00e+00 2.40e+01 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.50e+01 5.00e+00 5.62e+02 7.70e+01 0.00e+00]
  [0.00e+00 2.60e+01 5.00e+00 5.62e+02 7.70e+01 0.00e+00]]

 [[0.00e+00 1.00e+00 5.00e+00 8.39e+02 7.30e+01 0.00e+00]
  [0.00e+00 2.00e+00 5.00e+00 8.39e+02 7.30e+01 0.00e+00]
  [1.00e+00 3.00e+00 5.00e+00 8.39e+02 7.30e+01 3.30e+03]
  ...
  [0.00e+00 2.40e+01 5.00e+00 8.39e+02 7.30e+01 0.00e+00]
  [0.00e+0

In [12]:
#normalise data

train_mean = train_arr.mean(axis=(0, 1))
train_std = train_arr.std(axis=(0, 1))
train_arr = (train_arr - train_mean)/train_std
print(train_arr)
val_arr = (val_arr - train_mean)/train_std


[[[-0.09934477 -1.66666667 -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  [-0.09934477 -1.53333333 -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  [-0.09934477 -1.4        -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  ...
  [-0.09934477  1.4        -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  [-0.09934477  1.53333333 -1.51708349  0.10151063  1.19542789
   -0.1184017 ]
  [-0.09934477  1.66666667 -1.51708349  0.10151063  1.19542789
   -0.1184017 ]]

 [[-0.09934477 -1.66666667 -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  [-0.09934477 -1.53333333 -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  [-0.09934477 -1.4        -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  ...
  [-0.09934477  1.4        -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  [-0.09934477  1.53333333 -1.51708349  0.1082948   1.19542789
   -0.1184017 ]
  [-0.09934477  1.66666667 -1.51708349  0.1082948   1.19542789
   -0.1184017 ]]

 [[-0.09934477 -1.66666667 -1.517083

In [13]:
#train_arr = list(train_arr)

#train_arr = train_arr[0:3]
train_list = list(train_arr)
val_list = list(val_arr)
first_test_list = list(train_arr_first_test)
#def split_sequences(sequences, n_steps):
#    pass

In [14]:
#get sliding windows for each ID
train_x, train_y = [],[]
val_x, val_y = [],[]
def get_windows(sequence, n_steps, x, y):
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix][0]
        x.append(seq_x)
        y.append(seq_y)

for example in train_list:
    get_windows(example, 6, train_x, train_y)
for example in val_list:
    get_windows(example, 6, val_x, val_y)
print(val_y[0])

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)
print(train_x.shape)
print(val_y.shape)

-0.09934476591470832
(147000, 6, 6)
(7350,)


In [62]:
shop_id_x = train_x[:, :, 2]
shop_id_val_x = val_x[:, :, 2]
train_x = np.delete(train_x, 2, axis=2)
print(shop_id_x)
input_dim = len(set(tuple(shop_id_x.flatten())))
shop_id_input = tf.keras.Input(shape=(6,1))
numerical_input = tf.keras.Input(shape=(6, 5))
shop_id_embed = tf.keras.layers.Embedding(input_dim=input_dim, output_dim=8)(shop_id_input)
shop_id_flatten = tf.reshape(shop_id_embed, shape=[147000, 6, 8])

combine_input = tf.keras.layers.Concatenate()([shop_id_flatten, numerical_input])
model_shop_id = tf.keras.Model([shop_id_input, numerical_input], combine_input)
print(combine_input.shape)

[[ 1.19542789  1.19542789  1.19542789  1.19542789  1.19542789  1.19542789]
 [ 1.19542789  1.19542789  1.19542789  1.19542789  1.19542789  1.19542789]
 [ 1.19542789  1.19542789  1.19542789  1.19542789  1.19542789  1.19542789]
 ...
 [-1.2477824  -1.2477824  -1.2477824  -1.2477824  -1.2477824  -1.2477824 ]
 [-1.2477824  -1.2477824  -1.2477824  -1.2477824  -1.2477824  -1.2477824 ]
 [-1.2477824  -1.2477824  -1.2477824  -1.2477824  -1.2477824  -1.2477824 ]]
(147000, 6, 13)


In [63]:
lstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(12))(combine_input)
dense_1 = tf.keras.layers.Dense(1)(lstm_1)
total_model = tf.keras.Model([shop_id_input, numerical_input], dense_1)
model.compile(loss="mse", optimizer="adam")
history = model.fit(x=[shop_id_x, train_x], y=train_y, validation_data=(val_x, val_y), epochs=10, batch_size=40, verbose=1)



Epoch 1/10


ValueError: in user code:

    e:\latest pycharm\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    e:\latest pycharm\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    e:\latest pycharm\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    e:\latest pycharm\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    e:\latest pycharm\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    e:\latest pycharm\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outpu

In [ ]:
#model = tf.keras.models.Sequential([
 #      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(12, input_shape=(6,6))),
 #   tf.keras.layers.Dense(1)
#])

#model.compile(loss="mse", optimizer="adam")
#history = model.fit(x=train_x, y=train_y, validation_data=(val_x, val_y), epochs=100, batch_size=40, verbose=1)


In [16]:
#print(val_arr)
#print(train_arr[:,-7:-1])
#create prediction window excluding last value to compare
predictions = model.predict(val_arr[:,:-1])
predictions = model.predict(train_arr[:,-7:-1])
denormalised_predictions = predictions*train_std[0] + train_mean[0]

UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/bidirectional/forward_lstm/PartitionedCall]] [Op:__inference_predict_function_2880]

Function call stack:
predict_function -> predict_function -> predict_function


In [ ]:

print(denormalised_predictions.astype(int))
print(train_mean)
print(train_std)